In [1]:
import sys
sys.path.append('/cephfs/home/lc865/workspace/similar-expressions/src/SR_benchmarking')

import dataset_utils
from importlib import reload
reload(dataset_utils)
import pandas as pd

In [2]:
data = dataset_utils.load_datasets('feynman', 1000, 1e-4, equation_indices=[2], forbid_ops=['log'])[0]
i = 1

print(data.equation)
print(data.X[:, i].min(), data.X[:, i].max())

f = exp(-(theta/sigma)**2/2)/(sqrt(2*pi)*sigma)
1.0001367930556775 2.99953825243722


In [3]:
from pysr import PySRRegressor, TensorBoardLoggerSpec
from pysr_interface_utils import get_mutation_stats, reset_mutation_stats

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling SymbolicRegression...
  36266.0 ms  ✓ SymbolicRegression
  1 dependency successfully precompiled in 37 seconds. 189 already precompiled.
Precompiling SymbolicRegressionJSON3Ext...
   3515.8 ms  ✓ SymbolicRegression → SymbolicRegressionJSON3Ext
  1 dependency successfully precompiled in 4 seconds. 195 already precompiled.


In [13]:
# Load model
custom_loss = """
    function eval_loss(tree, dataset::Dataset{T,L}, options)::L where {T,L}
        prediction, flag = eval_tree_array(tree, dataset.X, options)
        if !flag
            return L(Inf)
        end
        return sum( (1000 .* (prediction .- dataset.y) ) .^ 2) / dataset.n
    end
    """

early_stopping_condition = 1e-3

model_id = 'e51hcsb9'

logger_spec = TensorBoardLoggerSpec(
    log_dir="logs/run",
    log_interval=10,  # Log every 10 iterations
)

model = PySRRegressor(
    niterations=4,
    binary_operators=["+", "*", "-", "/"],
    unary_operators=["cos","exp","sin","zero_sqrt(x) = x >= 0 ? sqrt(x) : zero(x)"],
    extra_sympy_mappings={"zero_sqrt": lambda x: x},  # TODO: Not using Sympy rn. Fix this.
    precision=64,
    neural_options=dict(
        active=True,
        # model_path=f'/Users/luis/Desktop/Cranmer2024/Workplace/smallMutations/similar-expressions/src/dev/ONNX/onnx-models/model-{model_id}.onnx',
        model_path=f'/cephfs/home/lc865/workspace/similar-expressions/src/ONNX_conversion/onnx-models//model-{model_id}.onnx',
        sampling_eps=1e-3,
        subtree_min_nodes=2,
        subtree_max_nodes=14,
        device="cuda",
        verbose=True,
        max_tree_size_diff=7,
        require_tree_size_similarity=True,
        require_novel_skeleton=True,
        require_expr_similarity=True,
        similarity_threshold=0.2,
        max_resamples=127,
        sample_batchsize=32,
        sample_logits=True,
        log_subtree_strings=True,
        subtree_max_features=2
    ),
    weight_neural_mutate_tree=0.1,
    # elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    loss_function=custom_loss,
    early_stop_condition=f"f(loss, complexity) = (loss < {early_stopping_condition:e})",
    logger_spec=logger_spec,
    # populations=10,
    # population_size=10,
    # tournament_selection_n=5,
)

dataset = dataset_utils.load_datasets('feynman', num_samples=2000, noise=0, equation_indices=[35])[0]
dataset.equation
X, y = dataset.X, dataset.y
model.fit(X, y)

/Users/luis/opt/anaconda3/envs/simexpr-arm64/lib/python3.12/site-packages/pysr/sr.py:2785: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(



Expressions evaluated per second: 2.710e+04
Progress: 112 / 124 total iterations (90.323%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.997e+06  3.604e+01  y = x₁
3           5.557e+05  6.395e-01  y = x₁ * 1.3773
5           5.557e+05  -0.000e+00  y = (x₁ * 2.3773) - x₁
6           3.415e+05  4.868e-01  y = x₁ + (x₁ / zero_sqrt(x₂))
7           3.333e+05  2.438e-02  y = (x₀ / (x₂ / x₁)) + x₁
8           3.297e+05  1.102e-02  y = x₁ + (x₀ / zero_sqrt(x₂ / x₁))
10          1.807e+05  3.007e-01  y = x₁ - (((-2.7773 - x₁) / x₂) * zero_sqrt(x₀))
14          1.387e+05  6.618e-02  y = (exp(4.1779 / x₂) + (-2.1294 / x₀)) + exp(zero_sqrt(x₁...
                                       + -1.4762))
19          9.557e+04  7.442e-02  y = exp(zero_sqrt((-1.6696 + x₁) - (exp(x₀) * (-0.49502 / 

[ Info: Started!
[ Info: Initializing sampling model.
[ Info: ops: Function[+, *, -, /, cos, exp, sin, Main.zero_sqrt]
[ Info: Final population:
[ Info: Results saved to:


PySRRegressor.equations_ = [
	   pick         score                                           equation  \
	0        0.000000e+00                                                 x1   
	1        6.394535e-01                            x1 * 1.3773326571932893   
	2        1.110223e-16                     (x1 * 2.3773326571934987) - x1   
	3        4.868325e-01                          x1 + (x1 / zero_sqrt(x2))   
	4        2.438078e-02                              (x0 / (x2 / x1)) + x1   
	5        1.101520e-02                     x1 + (x0 / zero_sqrt(x2 / x1))   
	6        3.006940e-01  x1 - (((-2.7773263540213 - x1) / x2) * zero_sq...   
	7        1.475178e-01  exp(zero_sqrt(x1 + -1.5638591075632786)) + exp...   
	8  >>>>  1.466047e-01  exp(zero_sqrt(x1 - (((0.15760553428445512 - (x...   
	9        1.227405e-01  exp(zero_sqrt(x1 - ((((-0.19900083056300694 - ...   
	
	           loss  complexity  
	0  1.996601e+06           1  
	1  5.557367e+05           3  
	2  5.557367e+05           5  
	3  3.415391e+05           6  
	4  3.333128e+05           7  
	5  3.296614e+05           8  
	6  1.806711e+05          10  
	7  1.001438e+05          14  
	8  6.450824e+04          17  
	9  5.046661e+04          19  
]

  - outputs/20250418_160404_zs5fnC/hall_of_fame.csv


In [12]:
# import pysr

pysr.julia_import.SymbolicRegression.NeuralLoggingModule.init_logger('/Users/luis/Desktop/Cranmer2024/Workplace/smallMutations/similar-expressions/src/SR_benchmarking/SR_loss_logging/logs')


SymbolicRegression.NeuralLoggingModule.Logger("/Users/luis/Desktop/Cranmer2024/Workplace/smallMutations/similar-expressions/src/SR_benchmarking/SR_loss_logging/logs", "/Users/luis/Desktop/Cranmer2024/Workplace/smallMutations/similar-expressions/src/SR_benchmarking/SR_loss_logging/logs/sr_logs_2025-04-18_16-04-01.csv", @NamedTuple{timestamp::Float64, mutation_type::String, num_evals::Int64, attempts::Int64, loss_before::Float32, loss_after::Float32, score_before::Float32, score_after::Float32, successful_mutation::Bool, mutation_accepted::Bool, result_reason::String, TED::Float32}[#undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef  …  #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef], 10000, 0, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (2, 0, -1)))

In [14]:
pysr.julia_import.SymbolicRegression.NeuralLoggingModule.close_global_logger_b()